In [ ]:
import sys
import os
import cv2
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn
from utils import *
from sklearn.metrics import mean_squared_error

## For the train video

In [ ]:
train_speed = np.loadtxt('./data/train.txt')
total_frame = train_speed.shape[0]

cap = cv2.VideoCapture('./data/train.mp4')
if not cap.isOpened():
    print("Opening video failed")

In [ ]:
def process_video_img(img):
    img = crop_image(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

In [ ]:
model = nvidia_model()
model.load_weights('nvidia_1.h5')

In [ ]:
cap.set(cv2.CAP_PROP_POS_FRAMES,0)
ret,img1 = cap.read()
img1_gray = process_video_img(img1)
idx =1
font                   = cv2.FONT_HERSHEY_SIMPLEX
TrueSpeedLoc           = (10,70)
PredSpeedLoc           = (10,90)
MseSpeedLoc            = (10,110)
fontScale              = 0.5
lineType               = 2
all_true = []
all_pred = []
for idx in range(1,total_frame-1):
    cap.set(cv2.CAP_PROP_POS_FRAMES,idx)
    ret,img2 = cap.read()
    if ret:
        img2_gray = process_video_img(img2)
       
        flow = single_optical_flow_dense(img1_gray,img2_gray)
        flow = flow.reshape(1,flow.shape[0],flow.shape[1],flow.shape[2])
        predict_speed = model.predict(flow)
        
        predicted = predict_speed[0][0]
        trueLabel = train_speed[idx]
        mse = (predicted-trueLabel)**2
        all_true.append(trueLabel)
        all_pred.append(predicted)
        true_speed_text = 'True: ' + str(trueLabel)
        pred_speed_text = 'Pred:' + str(predicted)
        mse_text = 'mse:' + str(mse)
        cv2.putText(img2, true_speed_text, TrueSpeedLoc, font, fontScale,(255,0,0),lineType)
        cv2.putText(img2, pred_speed_text, PredSpeedLoc, font, fontScale,(0,255,0),lineType)
        cv2.putText(img2, mse_text, MseSpeedLoc, font, fontScale,(0,0,255),lineType)
        cv2.imshow('image',img2)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        img1=img2
        img1_gray = img2_gray

cv2.destroyAllWindows()

In [ ]:
mean_squared_error(all_true,all_pred)

## For the test video

In [ ]:
cap = cv2.VideoCapture('./data/test.mp4')
if not cap.isOpened():
    print("Opening video failed")
cap.set(cv2.CAP_PROP_POS_FRAMES,0)
ret,img1 = cap.read()
img1_gray = process_video_img(img1)
idx = 1
font                   = cv2.FONT_HERSHEY_SIMPLEX
predSpeedLoc           = (10,70)
fontScale              = 0.5
lineType               = 2
all_pred = []

while True:
    cap.set(cv2.CAP_PROP_POS_FRAMES,idx)
    ret,img2 = cap.read()
    if ret:
        img2_gray = process_video_img(img2)
       
        flow = single_optical_flow_dense(img1_gray,img2_gray)
        flow = flow.reshape(1,flow.shape[0],flow.shape[1],flow.shape[2])
        predict_speed = model.predict(flow)
        
        predicted = predict_speed[0][0]
        trueLabel = train_speed[idx]
        
        all_pred.append(predicted)
        pred_speed_text = 'Pred:' + str(predicted)
        cv2.putText(img2, pred_speed_text, predSpeedLoc, font, fontScale,(0,255,0),lineType)
       
        cv2.imshow('image',img2)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        img1=img2
        img1_gray = img2_gray
        idx += 1
    else:
        break

cv2.destroyAllWindows()